# Ocean Sciences 

## Homework #4

In [1]:
# imports
import numpy as np
from importlib import reload

from oceanpy import constants
from oceanpy import units
from oceanpy.forces import coriolis as ofc

from bokeh import plotting
from bokeh import models

In [2]:
def set_fontsize(p, fsz):
    '''
    Parameters
    ----------
    ax : Bokeh plot class
    fsz : float
      Font size
    '''
    p.xaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.xaxis.major_label_text_font_size = "{:d}pt".format(fsz)
    #
    p.yaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.yaxis.major_label_text_font_size = "{:d}pt".format(fsz)

# 1 Coriolis Acceleration

## a) Coriolis parameter

### $f = 2 \Omega \, \sin\phi$

### Santa Cruz

In [3]:
f_SC = ofc.coriolis_parameter(37*units.deg)
f_SC

<Quantity 8.77701897e-05 1 / s>

### Range of Latitudes

In [4]:
phis = np.linspace(-90., 90., 180)*units.deg
f = ofc.coriolis_parameter(phis)

In [5]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Coriolis Parameter', x_axis_label='Latitude [deg]',
                   y_axis_label='f [1/s]')

p.line(phis, f, legend='f(phi)', line_width=2)
p.legend.location = "bottom_left"

set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### It is a maximum at +90 deg (90N) and a minimum at -90deg (90S).

### It is zero at 0 deg.

## b) Meridional derivative

### $\beta = \partial f / \partial y = \frac{1}{R} \frac{\partial f}{\partial \phi}$

### $\beta = \frac{2 \Omega}{R}  \, \cos\phi$

In [6]:
beta = ofc.meridional_derivative(phis)
beta

<Quantity [1.40014671e-24, 4.01298220e-10, 8.02472831e-10, 1.20340026e-09,
           1.60395702e-09, 2.00401972e-09, 2.40346514e-09, 2.80217024e-09,
           3.20001220e-09, 3.59686849e-09, 3.99261686e-09, 4.38713542e-09,
           4.78030264e-09, 5.17199742e-09, 5.56209911e-09, 5.95048754e-09,
           6.33704309e-09, 6.72164669e-09, 7.10417987e-09, 7.48452480e-09,
           7.86256433e-09, 8.23818201e-09, 8.61126215e-09, 8.98168982e-09,
           9.34935093e-09, 9.71413223e-09, 1.00759214e-08, 1.04346069e-08,
           1.07900783e-08, 1.11422261e-08, 1.14909419e-08, 1.18361182e-08,
           1.21776487e-08, 1.25154283e-08, 1.28493527e-08, 1.31793193e-08,
           1.35052264e-08, 1.38269736e-08, 1.41444617e-08, 1.44575930e-08,
           1.47662711e-08, 1.50704008e-08, 1.53698885e-08, 1.56646419e-08,
           1.59545703e-08, 1.62395843e-08, 1.65195961e-08, 1.67945195e-08,
           1.70642698e-08, 1.73287640e-08, 1.75879205e-08, 1.78416595e-08,
           1.80899029e-08

In [7]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Meridional derivative', x_axis_label='Latitude [deg]',
                                       y_axis_label='beta [1/km s]')


p.line(phis, beta, legend='beta(phi)', line_width=2)
p.legend.location = "top_left"

set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### $\beta$ is positive everywhere except $\phi = 90$deg where it is zero.

## c) Acceleration

### $a = d^2x/dt^2$

### Integrate once:  $v(t) = at + v_0$

### Integrate twice:  $x(t) = at^2/2 + v_0 t + x_0$

## d) Baseball

### Imperial to SI

In [16]:
v_0 = 80 * units.imperial.mi / units.hour
v_0.cgs.to('km/hr')

<Quantity 128.74752 km / h>

In [15]:
d = 60 * units.imperial.foot
d.cgs.to('m')

<Quantity 18.288 m>

### Coriolis acceleration

### $f v$

In [17]:
# Assume Santa Cruz
f = ofc.coriolis_parameter(37*units.deg)

In [19]:
a_c = f * v_0
a_c.to('m/s**2')

<Quantity 0.00313894 m / s2>

### Time to plate

In [21]:
t = d/v_0
t.to('s')

<Quantity 0.51136364 s>

### Deflection

In [23]:
x = a_c * t**2 / 2
x.to('cm')

<Quantity 0.04104054 cm>

## e) Airplane

In [27]:
v_plane = 250 * units.m / units.s  # East
phi = -26*units.deg

### Coriolis

In [28]:
f_26S = ofc.coriolis_parameter(phi)
f_26S

<Quantity -6.3933131e-05 1 / s>

In [29]:
a_plane = f_26S * v_plane
a_plane.to('m/s**2')

<Quantity -0.01598328 m / s2>

#### Direction is to the left in the Southern Hemisphere, i.e. North

In [31]:
# Deflection
x_coriolis = a_plane * (1*units.hour)**2 / 2
x_coriolis.to('km')

<Quantity -103.57167218 km>

### Compare

----

# Tests

In [ ]:
units.mile